### En este notebook encontrarás el examen de desarrollo en pySpark, respira profundo y manos a la obra. Si te bloqueas puedes salir y respirar aire fresco, realiza alguna otra actividad y retoma el examen en cuanto puedas. Mucho éxito.

#### Ejecutamos requerimientos

In [ ]:
%run ../requirements/requirements.py

In [ ]:
from IPython.core.display import HTML
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql import Window

display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
spark

#### Lectura de tablas

In [ ]:
AIRLINES_DF = spark.read.parquet("../resources/data/exam/parquet/airlines")
AIRPORTS_DF = spark.read.parquet("../resources/data/exam/parquet/airports")
FLIGHTS_DF = spark.read.parquet("../resources/data/exam/parquet/flights")

#### Transformaciones Narrow

##### De acuerdo a la tabla FLIGHTS_DF el cliente ha solicitado generar algunas columnas que aportan mucho valor. Resuelve las siguientes reglas de negocio, apoyate de los ejemplos mostrados.

1. Generar una columna llamada `DATE_FLIGHT_TRIP` (fecha del vuelo) la cual contendrá la concatenación de las columnas `YEAR`, `MONTH` y `DAY`.
    El formato de esta columna resultante será: `yyyy-MM-dd` y el tipo de dato será **DateType**.
    ##### Ejemplo:
    ```
            +----+-----+---+----------------+
            |YEAR|MONTH|DAY|DATE_FLIGHT_TRIP|
            +----+-----+---+----------------+
            |2015|   10|  9|      2015-10-09|
            |2015|    1|  4|      2015-01-04|
            |2015|    2| 26|      2015-02-26|
            +----+-----+---+----------------+
    ```
2. Genera una columna de tipo **IntegerType** llamada `DAY_OF_WEEK` (día de la semana del vuelo) la cual contendrá el día de la semana del campo `DATE_FLIGHT_TRIP`, esta columna contendrá valores numéricos del 1 al 7 de acuerdo al siguiente criterio:
    ```
            +-----+--------------------------------------+
            |valor| criterio                             |
            +-----|--------------------------------------+
            | 1   | si el día de la semana fue lunes     |
            | 2   | si el día de la semana fue martes    |
            | 3   | si el día de la semana fue miercoles |
            | 4   | si el día de la semana fue jueves    |
            | 5   | si el día de la semana fue viernes   |
            | 6   | si el día de la semana fue sabado    |
            | 7   | si el día de la semana fue domingo   |
            +-----+--------------------------------------+
    ```
    ##### Ejemplo:
    ```
            +----------------+-----------+
            |DATE_FLIGHT_TRIP|DAY_OF_WEEK|
            +----------------+-----------+
            |      2015-09-06|          7|
            |      2015-11-24|          2|
            |      2015-11-26|          4|
            +----------------+-----------+
    ```
3. Convierte la columna `SCHEDULED_DEPARTURE` (hora de salida prevista) la cual contiene la hora y minutos (`HHmm`) a una representación con el formato `yyyy-MM-dd HH:mm:ss` y de tipo **TimestampType**, la fecha se obtendrá a partir de la columna `DATE_FLIGHT_TRIP` y se colocarán los segundos en `00`.
   NOTA: Cuando el valor original de la columna `SCHEDULED_DEPARTURE` sea `2400`, se deberá registrar como hora `00` y minuto `00`.
    ##### Ejemplo:
    ```
            +----------------+------------------------------+---------------------------+
            |DATE_FLIGHT_TRIP|SCHEDULED_DEPARTURE (anterior)|SCHEDULED_DEPARTURE (nuevo)|
            +----------------+------------------------------+---------------------------+
            |      2015-10-28|                          1005|        2015-10-28 10:05:00|
            |      2015-08-29|                          1000|        2015-08-29 10:00:00|
            |      2015-06-21|                          1350|        2015-06-21 13:50:00|
            +----------------+------------------------------+---------------------------+
    ```
4. Genera la columna `SCHEDULED_ARRIVAL` (fecha de llegada prevista) con el formato `yyyy-MM-dd HH:mm:ss` y de tipo **TimestampType**, la fecha se obtendrá de la siguiente manera: `SCHEDULED_ARRIVAL = SCHEDULED_DEPARTURE + SCHEDULED_TIME + (DIFF_TIME_ZONE * 60)`. Esto es a la fecha dada en `SCHEDULED_DEPARTURE` se le suma la cantidad de MINUTOS dados por `SCHEDULED_TIME + (DIFF_TIME_ZONE * 60)`
    ##### Ejemplo:
    ```
            +-------------------+--------------+--------------+-------------------+
            |SCHEDULED_DEPARTURE|SCHEDULED_TIME|DIFF_TIME_ZONE|  SCHEDULED_ARRIVAL|
            +-------------------+--------------+--------------+-------------------+
            |2015-01-01 06:55:00|           175|             1|2015-01-01 10:50:00|
            |2015-01-01 08:45:00|           101|             1|2015-01-01 11:26:00|
            |2015-01-01 13:15:00|           152|            -1|2015-01-01 14:47:00|
            +-------------------+--------------+--------------+-------------------+
    ```
5. Genera la columna `DEPARTURE_TIME` (tiempo de salida del vuelo) con formato lógico `yyyy-MM-dd HH:mm:ss` y de tipo **TimestampType**, esta columna se calcula de la siguiente manera:
    * Si `CANCELLED == 1` entonces `DEPARTURE_TIME` es NULL
    * En otro caso `DEPARTURE_TIME` se calcula sumando la cantidad de MINUTOS representados por `DEPARTURE_DELAY` a la columna `SCHEDULED_DEPARTURE`: `DEPARTURE_TIME = SCHEDULED_DEPARTURE + DEPARTURE_DELAY`
    ##### Ejemplo:
    ```
            +---------+-------------------+---------------+-------------------+
            |CANCELLED|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|     DEPARTURE_TIME|
            +---------+-------------------+---------------+-------------------+
            |        1|2015-01-27 09:00:00|           null|               null|
            |        0|2015-09-11 08:40:00|             21|2015-09-11 09:01:00|
            |        0|2015-04-29 07:00:00|             -3|2015-04-29 06:57:00|
            |        0|2015-08-28 07:00:00|              1|2015-08-28 07:01:00|
            +---------+-------------------+---------------+-------------------+
    ```
6. Genera la columna `WHEELS_OFF` (el momento en que las ruedas del avión dejan el suelo) en el formato lógico `yyyy-MM-dd HH:mm:ss` y de tipo **TimestampType**, esta columna se genera de la siguiente manera:
    * Si `CANCELLED == 1` entonces `WHEELS_OFF` es NULL
    * En otro caso `WHEELS_OFF` se obtendrá a partir de la suma de MINUTOS de `TAXI_OUT` a la columna `DEPARTURE_TIME`: `WHEELS_OFF = DEPARTURE_TIME + TAXI_OUT`
    ##### Ejemplo:
    ```
            +---------+-------------------+--------+-------------------+
            |CANCELLED|     DEPARTURE_TIME|TAXI_OUT|         WHEELS_OFF|
            +---------+-------------------+--------+-------------------+
            |        1|               null|    null|               null|
            |        0|2015-09-11 09:01:00|       8|2015-09-11 09:09:00|
            |        0|2015-05-08 14:43:00|      12|2015-05-08 14:55:00|
            +---------+-------------------+--------+-------------------+
    ```
7. Genera la columna `ARRIVAL_TIME` (tiempo de llegada con formato lógico `yyyy-MM-dd HH:mm:ss` y de tipo **TimestampType)**, esta columna se calcula de la siguiente manera:
    * Si `DIVERTED == 1` entonces `ARRIVAL_TIME` es NULL
    * En otro caso `ARRIVAL_TIME` se calcula sumando la cantidad de MINUTOS de `ARRIVAL_DELAY` a la columna `SCHEDULED_ARRIVAL`: `ARRIVAL_TIME = SCHEDULED_ARRIVAL + ARRIVAL_DELAY`
    ##### Ejemplo:
    ```
            +--------+-------------------+-------------+-------------------+
            |DIVERTED|  SCHEDULED_ARRIVAL|ARRIVAL_DELAY|       ARRIVAL_TIME|
            +--------+-------------------+-------------+-------------------+
            |       0|2015-09-11 09:55:00|           45|2015-09-11 10:40:00|
            |       0|2015-04-29 08:05:00|          -11|2015-04-29 07:54:00|
            |       1|2015-05-27 08:05:00|         null|               null|
            +--------+-------------------+-------------+-------------------+
    ```
8. Genera la columna `WHEELS_ON` (el momento en que las ruedas del avión tocan el suelo) con formato lógico `yyyy-MM-dd HH:mm:ss` y de tipo **TimestampType**, esta columna se actualiza de la siguiente manera:
    * Si `DIVERTED == 1` entonces `WHEELS_ON` es NULL
    * En otro caso `WHEELS_ON` se obtendrá a partir de la resta de MINUTOS de `TAXI_IN` a la columna `ARRIVAL_TIME`: `WHEELS_ON = ARRIVAL_TIME - TAXI_IN`
    ##### Ejemplo:
    ```
            +--------+-------------------+-------+-------------------+
            |DIVERTED|       ARRIVAL_TIME|TAXI_IN|          WHEELS_ON|
            +--------+-------------------+-------+-------------------+
            |       1|               null|   null|               null|
            |       0|2015-09-11 10:40:00|     26|2015-09-11 10:14:00|
            |       0|2015-08-28 07:58:00|      7|2015-08-28 07:51:00|
            +--------+-------------------+-------+-------------------+
    ```
9. Genera la columna `AIR_TIME` de tipo **IntegerType**, la cual contiene la duración en minutos del tiempo en vuelo, la operación a realizar es la siguiente: `TRAVEL_TIME - DIFF_TIME_ZONE*60`, donde `TRAVEL_TIME` representa la diferencia de tiempo entre `WHEELS_ON` y `WHEELS_OFF`
    ##### Ejemplo:
    ```
            +-------------------+-------------------+--------------+--------+
            |          WHEELS_ON|         WHEELS_OFF|DIFF_TIME_ZONE|AIR_TIME|
            +-------------------+-------------------+--------------+--------+
            |2015-09-11 10:14:00|2015-09-11 09:09:00|            -2|     185|
            |2015-04-29 07:50:00|2015-04-29 07:06:00|             0|      44|
            |2015-10-06 16:12:00|2015-10-06 08:23:00|             3|     289|
            +-------------------+-------------------+--------------+--------+
    ```
10. Genera la columna `ELAPSED_TIME` de tipo **IntegerType**, la cual contiene el tiempo transcurrido entre la puerta del aeropuerto origen y la puerta del aeropuerto destino, esta columna se obtiene de calcular la suma de las columnas: `AIR_TIME + TAXI_IN + TAXI_OUT`
    ##### Ejemplo:
    ```
            +--------+-------+--------+------------+
            |AIR_TIME|TAXI_IN|TAXI_OUT|ELAPSED_TIME|
            +--------+-------+--------+------------+
            |     351|      6|      13|         370|
            |     306|      4|      19|         329|
            +--------+-------+--------+------------+
    ```
11. Reemplaza los valores NULL de las columnas `WEATHER_DELAY`, `LATE_AIRCRAFT_DELAY`, `AIRLINE_DELAY`, `SECURITY_DELAY`, `AIR_SYSTEM_DELAY`, `ARRIVAL_DELAY`, `DEPARTURE_DELAY` por el valor numérico 0

Almacena la salida de las 11 reglas de negocio en la variable **narrow_flights_df**, asegurate de que tu tabla tenga la siguiente estructura:
```
|-- ID: string
|-- DATE_FLIGHT_TRIP: date
|-- DAY_OF_WEEK: integer
|-- ORIGIN_AIRPORT: string
|-- DESTINATION_AIRPORT: string
|-- DISTANCE: double
|-- AIRLINE: string
|-- SCHEDULED_DEPARTURE: timestamp
|-- SCHEDULED_ARRIVAL: timestamp
|-- SCHEDULED_TIME: integer
|-- DIFF_TIME_ZONE: integer
|-- CANCELLED: integer
|-- CANCELLATION_REASON: string
|-- DIVERTED: integer
|-- DEPARTURE_TIME: timestamp
|-- WHEELS_OFF: timestamp
|-- AIR_TIME: integer
|-- WHEELS_ON: timestamp
|-- ARRIVAL_TIME: timestamp
|-- ELAPSED_TIME: integer
|-- WEATHER_DELAY: integer
|-- LATE_AIRCRAFT_DELAY: integer
|-- AIRLINE_DELAY: integer
|-- SECURITY_DELAY: integer
|-- AIR_SYSTEM_DELAY: integer
|-- ARRIVAL_DELAY: integer
|-- DEPARTURE_DELAY: integer
```

In [ ]:
# -------------------------------------------------------------------------------------------------------------- #
# ----------------------------------- TUS TRANSFORMACIONES VAN EN ESTA CELDA ----------------------------------- #
# -------------------------- Almacena tu resultado en la variable "narrow_flights_df" -------------------------- #
# -------------------------------------------------------------------------------------------------------------- #

narrow_flights_df = FLIGHTS_DF # Agrega tus transformaciones

In [ ]:
# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------ VALIDACIÓN ------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #
# -------------------------------------------- NO MODIFICAR CELDA -------------------------------------------- #
# ------------------------------------------------------------------------------------------------------------ #

validation_narrow_transformations(narrow_flights_df)

#### Transformaciones Wide

##### Utilizando el resultado obtenido en narrow_flights_df el cliente ha solicitado agregar algunas columnas adicionales y eliminar algunos registros que considera innecesarios. El resultado de las transformaciones se almacenará en la variable wide_flights_df

1. Agrega una columna llamada `TOP_AIRLINE_ARRIVAL` de tipo **IntegerType** donde se colocará el valor top (RANK) de cada aerolinea con mayor porcentaje de llegadas a tiempo en sus vuelos, esta columna se obtendrá de la siguiente forma:
    1. Obtener el total de vuelos por cada aerolinea.
    2. Obtener el total de vuelos no desviados `DIVERTED = 0` y con un retraso menor o igual a cero `ARRIVAL_DELAY <= 0` por cada aerolinea.
    3. Obtener el porcentaje de llegadas a tiempo, esto es divide el resultado del paso 2 entre el resultado del paso 1.
    4. Genera la columna `TOP_AIRLINE_ARRIVAL` utilizando la función RANK sobre el resultado del paso 3, donde el valor 1 de la función RANK corresponde al porcentaje más alto (mayor índice de llegadas a tiempo).
    5. Agrega la columna `TOP_AIRLINE_ARRIVAL` al dataframe original narrow_flights_df
        ##### Salida esperada para la columna TOP_AIRLINE_ARRIVAL
        ```
        +-------+-------+------+------------------+-------------------+
        |AIRLINE| step_1|step_2|            step_3|TOP_AIRLINE_ARRIVAL|
        +-------+-------+------+------------------+-------------------+
        |     DL| 875881|623259|0.7115795410563763|                  1|
        |     AS| 172521|115155|0.6674839584746205|                  2|
        |     AA| 725984|471663|0.6496878719090228|                  3|
        |     MQ| 294632|190311|0.6459278014608053|                  4|
        |     UA| 515723|328108|0.6362097482563314|                  5|
        |     WN|1261855|787679|0.6242230684191131|                  6|
        |     EV| 571977|356766|0.6237418637462695|                  7|
        |     OO| 588352|364339|0.6192534401174807|                  8|
        |     B6| 267048|164320|0.6153200922680567|                  9|
        |     US| 198715|122005|0.6139697556802456|                 10|
        |     VX|  61901| 37600|0.6074215279236199|                 11|
        |     HA|  76272| 46033|0.6035373400461507|                 12|
        |     F9|  90824| 49446|0.5444155729763058|                 13|
        |     NK| 117372| 60304|  0.51378522986743|                 14|
        +-------+-------+------+------------------+-------------------+
        ```
2. Elimina los registros donde las columnas `ORIGIN_AIRPORT` o `DESTINATION_AIRPORT` NO contengan algún valor valido registrado en el dataframe **AIRPORTS_DF** en la columna `AIRPORT_CODE`.
3. Agrega la columna `FLIGHT_NUMBER` de tipo **StringType** donde esta columna se generará a través de la concatenación de: `AIRLINE-ORIGIN_AIRPORT-DESTINATION_AIRPORT-FLIGHT_POSITION-REPEATED_ID`.
Las columnas `FLIGHT_POSITION` y `REPEATED_ID` se generan de la siguiente forma:
    - `FLIGHT_POSITION` se genera a traves de la función DENSE_RANK particionando por `DATE_FLIGHT_TRIP`, `ORIGIN_AIRPORT`, `DESTINATION_AIRPORT`, `AIRLINE` y ordenando de forma ascendente a través de la columna `SCHEDULED_DEPARTURE`
    - `REPEATED_ID` se genera a traves de la función ROW_NUMBER particionando por `DATE_FLIGHT_TRIP`, `ORIGIN_AIRPORT`, `DESTINATION_AIRPORT`, `AIRLINE`, `SCHEDULED_DEPARTURE` y ordenando de forma ascendente a través de la columna `SCHEDULED_DEPARTURE`. Los valores generados con ROW_NUMBER deberán ser modificados de acuerdo al siguiente diccionario: `{"1":"A","2":"B","3":"C"}`
      
        ##### Salida esperada para la columna FLIGHT_NUMBER
        ```
        +----------------+-------+--------------+-------------------+-------------------+---------------+
        |DATE_FLIGHT_TRIP|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|  FLIGHT_NUMBER|
        +----------------+-------+--------------+-------------------+-------------------+---------------+
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 07:41:00| OO-LAX-SAN-1-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 08:00:00| OO-LAX-SAN-2-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 09:40:00| OO-LAX-SAN-3-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 10:20:00| OO-LAX-SAN-4-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 10:40:00| OO-LAX-SAN-5-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 11:35:00| OO-LAX-SAN-6-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 11:40:00| OO-LAX-SAN-7-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 12:55:00| OO-LAX-SAN-8-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 13:00:00| OO-LAX-SAN-9-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 14:19:00|OO-LAX-SAN-10-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 14:25:00|OO-LAX-SAN-11-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 15:35:00|OO-LAX-SAN-12-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 16:05:00|OO-LAX-SAN-13-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 17:05:00|OO-LAX-SAN-14-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 17:05:00|OO-LAX-SAN-14-B|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 18:20:00|OO-LAX-SAN-15-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 19:29:00|OO-LAX-SAN-16-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 21:00:00|OO-LAX-SAN-17-A|
        |      2015-01-01|     OO|           LAX|                SAN|2015-01-01 23:10:00|OO-LAX-SAN-18-A|
        +----------------+-------+--------------+-------------------+-------------------+---------------+
        ```
Almacena la salida de tus transformaciones en la variable **wide_flights_df**, asegurate de que tu tabla contenga las columnas adicionales:
```
|-- TOP_AIRLINE_ARRIVAL: integer
|-- FLIGHT_NUMBER: string
```

In [ ]:
# ------------------------------------------------------------------------------------------------------------ #
# ---------------------------------- TUS TRANSFORMACIONES VAN EN ESTA CELDA ---------------------------------- #
# -------------------------- Almacena tu resultado en la variable "wide_flights_df" -------------------------- #
# ------------------------------------------------------------------------------------------------------------ #

wide_flights_df = narrow_flights_df # Agrega tus transformaciones

In [ ]:
# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------ VALIDACIÓN ------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #
# -------------------------------------------- NO MODIFICAR CELDA -------------------------------------------- #
# ------------------------------------------------------------------------------------------------------------ #

validation_wide_transformations(wide_flights_df)

#### Consultas adicionales

##### Utilizando el resultado obtenido en wide_flights_df el cliente ha solicitado realizar algunas consultas. Resuelve cada consulta y almacenalo en la variable que se te indica.

1. Cuales son las 3 aerolineas más eficientes en hora de llegada? El top 3 lo encuentras en la columna `TOP_AIRLINE_ARRIVAL`.

    ##### Almacena tu resultado en la variable airlines_top_3_df.

    ##### Salida esperada:
```
+-------+-------------------+----------------------+
|AIRLINE|TOP_AIRLINE_ARRIVAL|AIRLINE_NAME          |
+-------+-------------------+----------------------+
|DL     |1                  |Delta Air Lines Inc.  |
|AS     |2                  |Alaska Airlines Inc.  |
|AA     |3                  |American Airlines Inc.|
+-------+-------------------+----------------------+
```

In [ ]:
# ------------------------------------------------------------------------------------------------------------------ #
# ------------------------------------- TUS TRANSFORMACIONES VAN EN ESTA CELDA ------------------------------------- #
# ---------------------------- Almacena tu resultado en la variable "airlines_top_3_df" ---------------------------- #
# ------------------------------------------------------------------------------------------------------------------ #

airlines_top_3_df = wide_flights_df # Agrega tus transformaciones

In [ ]:
# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------ VALIDACIÓN ------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #
# -------------------------------------------- NO MODIFICAR CELDA -------------------------------------------- #
# ------------------------------------------------------------------------------------------------------------ #

validation_query_1(airlines_top_3_df)

2. Filtrando los registros que tengan retraso en salida (`DEPARTURE_DELAY`) mayor a 30 o retraso en llegada (`ARRIVAL_DELAY`) mayor a 30, para la fecha del vuelo (`DATE_FLIGHT_TRIP`) entre los días 2015-09-15 y 2015-12-15 y tomando únicamente las aerolineas (`AIRLINE`) DL, AS y AA. ¿Cuál es el valor promedio de las columnas `WEATHER_DELAY`, `LATE_AIRCRAFT_DELAY`, `AIRLINE_DELAY`, `SECURITY_DELAY` y `AIR_SYSTEM_DELAY`? Redondea tu resultado utilizando dos decimales.

    ##### Almacena tu resultado en la variable airlines_avg_delay_df.

    ##### Salida esperada:
```
+-------+-----------------+-----------------------+-----------------+------------------+--------------------+
|AIRLINE|AVG_WEATHER_DELAY|AVG_LATE_AIRCRAFT_DELAY|AVG_AIRLINE_DELAY|AVG_SECURITY_DELAY|AVG_AIR_SYSTEM_DELAY|
+-------+-----------------+-----------------------+-----------------+------------------+--------------------+
|     AA|             4.18|                  25.34|            29.21|              0.28|               13.56|
|     AS|             2.63|                   25.6|             23.0|              0.41|               14.48|
|     DL|             3.27|                  19.69|            38.85|               0.1|               12.68|
+-------+-----------------+-----------------------+-----------------+------------------+--------------------+
```

In [ ]:
# ------------------------------------------------------------------------------------------------------------------ #
# ------------------------------------- TUS TRANSFORMACIONES VAN EN ESTA CELDA ------------------------------------- #
# -------------------------- Almacena tu resultado en la variable "airlines_avg_delay_df" -------------------------- #
# ------------------------------------------------------------------------------------------------------------------ #

airlines_avg_delay_df = wide_flights_df # Agrega tus transformaciones

In [ ]:
# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------ VALIDACIÓN ------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #
# -------------------------------------------- NO MODIFICAR CELDA -------------------------------------------- #
# ------------------------------------------------------------------------------------------------------------ #

validation_query_2(airlines_avg_delay_df)

3. Para los días Viernes, Sabado y Domingo (`DAY_OF_WEEK IN (5,6,7)`) y omitiendo vuelos cancelados (`CANCELLED == 0`), ¿Cuál es el TOP 5 de Aeropuertos que presentan mayor afluencia de salidas (`ORIGIN_AIRPORT`) y llegadas (`DESTINATION_AIRPORT`)? Agrega las columnas donde se indiquen las ciudades y la cantidad de salidas y llegadas por cada posición del TOP 5.

    ##### Almacena tu resultado en la variable top_5_df.

    ##### Salida esperada:
```
+-----------+---+--------------------+-----------------+---------------+--------------------+-----------------+-------------+
|DAY_OF_WEEK|TOP|   DEPARTURE_AIRPORT|   DEPARTURE_CITY|DEPARTURE_COUNT|     ARRIVAL_AIRPORT|     ARRIVAL_CITY|ARRIVAL_COUNT|
+-----------+---+--------------------+-----------------+---------------+--------------------+-----------------+-------------+
|     Friday|  1|Hartsfield-Jackso...|          Atlanta|          50406|Hartsfield-Jackso...|          Atlanta|        50376|
|     Friday|  2|Chicago O'Hare In...|          Chicago|          41198|Chicago O'Hare In...|          Chicago|        41127|
|     Friday|  3|Dallas/Fort Worth...|Dallas-Fort Worth|          34024|Dallas/Fort Worth...|Dallas-Fort Worth|        34003|
|     Friday|  4|Denver Internatio...|           Denver|          28393|Denver Internatio...|           Denver|        28349|
|     Friday|  5|Los Angeles Inter...|      Los Angeles|          28078|Los Angeles Inter...|      Los Angeles|        28094|
|   Saturday|  1|Hartsfield-Jackso...|          Atlanta|          40339|Hartsfield-Jackso...|          Atlanta|        40501|
|   Saturday|  2|Chicago O'Hare In...|          Chicago|          32155|Chicago O'Hare In...|          Chicago|        32324|
|   Saturday|  3|Dallas/Fort Worth...|Dallas-Fort Worth|          29126|Dallas/Fort Worth...|Dallas-Fort Worth|        29189|
|   Saturday|  4|Los Angeles Inter...|      Los Angeles|          23865|Los Angeles Inter...|      Los Angeles|        23878|
|   Saturday|  5|Denver Internatio...|           Denver|          23131|Denver Internatio...|           Denver|        23301|
|     Sunday|  1|Hartsfield-Jackso...|          Atlanta|          49190|Hartsfield-Jackso...|          Atlanta|        49069|
|     Sunday|  2|Chicago O'Hare In...|          Chicago|          38955|Chicago O'Hare In...|          Chicago|        38706|
|     Sunday|  3|Dallas/Fort Worth...|Dallas-Fort Worth|          33042|Dallas/Fort Worth...|Dallas-Fort Worth|        32846|
|     Sunday|  4|Denver Internatio...|           Denver|          27871|Denver Internatio...|           Denver|        27704|
|     Sunday|  5|Los Angeles Inter...|      Los Angeles|          27404|Los Angeles Inter...|      Los Angeles|        27360|
+-----------+---+--------------------+-----------------+---------------+--------------------+-----------------+-------------+
```

In [ ]:
# --------------------------------------------------------------------------------------------------------- #
# -------------------------------- TUS TRANSFORMACIONES VAN EN ESTA CELDA --------------------------------- #
# ---------------------------- Almacena tu resultado en la variable "top_5_df" ---------------------------- #
# --------------------------------------------------------------------------------------------------------- #

top_5_df = wide_flights_df # Agrega tus transformaciones

In [ ]:
# ------------------------------------------------------------------------------------------------------------ #
# ------------------------------------------------ VALIDACIÓN ------------------------------------------------ #
# ------------------------------------------------------------------------------------------------------------ #
# -------------------------------------------- NO MODIFICAR CELDA -------------------------------------------- #
# ------------------------------------------------------------------------------------------------------------ #

validation_query_3(top_5_df)